In [ ]:
pip install pypdf

In [ ]:
pip install langchain_community

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
!pip install langchain

In [ ]:
loader = PyPDFLoader(r"/content/drive/MyDrive/Income Tax.pdf")
pages = loader.load_and_split()

In [ ]:
pages

In [ ]:
type(pages)

list

In [ ]:
len(pages)

1059

In [ ]:
!pip install --upgrade langchain_community
!pip install --upgrade langchain

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

In [ ]:
pip install sentence_transformers

In [ ]:
from langchain.text_splitter import SpacyTextSplitter

text_splitter = SpacyTextSplitter(chunk_size=500)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
separator = "\n",
chunk_size = 500,
chunk_overlap = 100,
length_function = len,
is_separator_regex = False,
)

In [ ]:
texts = text_splitter.split_documents(documents=pages)

In [ ]:
print(texts[0])

page_content='1 
THE INCOME-TAX ACT, 1961 
____________ 
ARRANGEMENT OF SECTIONS 
____________ 
CHAPTER I 
PRELIMINARY 
SECTIONS  
1. Short title, extent and commencement. 
2. Definitions. 
3. “Previous year” defined. 
CHAPTER II 
BASIS OF CHARGE 
4. Charge of income-tax. 
5. Scope of total income. 
5A. Apportionment of income between spouses governed by Portuguese Civil Code. 
6. Residence in India. 
7. Income deemed to be received. 
8. Dividend income. 
9. Income deemed to accrue or arise in India.' metadata={'source': '/content/drive/MyDrive/Income Tax.pdf', 'page': 0}


In [ ]:
print(len(texts))

7909


In [ ]:
type(texts)

list

In [ ]:
pip install sentence-transformers langchain chromadb

In [ ]:
pip install -U langchain-huggingface

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
!pip uninstall -y tensorflow transformers
!pip install tensorflow-cpu==2.12.0
!pip install transformers==4.33.2
!pip uninstall -y tensorflow
!pip install torch torchvision torchaudio
!pip install transformers
!pip uninstall -y tensorflow
!pip install tensorflow-cpu

In [ ]:
pip install sentence-transformers

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
vectorstore = Chroma.from_documents(texts, embedding_model)

In [ ]:
print(f"Created a vectorstore with {len(texts)} documents.")

Created a vectorstore with 7909 documents.


In [ ]:
pip install langchain chromadb sentence-transformers torch torchvision torchaudio transformers accelerate

In [ ]:
from huggingface_hub import login
login(token="YOUR_HUGGINGFACE_TOKEN")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

In [ ]:
def retrieve_context(question, retriever, top_k=5):
    """
    Retrieve the top-k most relevant documents from the vector store.
    """
    docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in docs[:top_k]])
    return context


In [ ]:
def generate_answer_with_mistral(question, context, model, tokenizer):
    """
    Use Mistral 7B to generate an answer based on the retrieved context.
    """
    prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)

    # Generating response
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.split("Answer:")[-1].strip()


In [ ]:
def rag_pipeline(question, retriever, model, tokenizer, top_k=5):
    """
    RAG pipeline: retrieve context and generate an answer.
    """
    # Step 1: Retrieving relevant context
    context = retrieve_context(question, retriever, top_k)

    if not context:
        return "Sorry, I couldn't find relevant information in the documents."

    # Step 2: Generating answer using Mistral 7B
    answer = generate_answer_with_mistral(question, context, model, tokenizer)
    return answer


In [ ]:
from langchain.vectorstores import Chroma

# Loading the Chroma vector store
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

question = "What are the general provisions mentioned in the Act?"

# Get the response
answer = rag_pipeline(question, retriever, model, tokenizer)
print(f"Question: {question}")
print(f"Answer: {answer}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Question: What are the general provisions mentioned in the Act?
Answer: 1. The provisions relating to the levy and collection of tax.
2. The provisions relating to the assessment and collection of tax.
3. The provisions relating to the determination of taxable income.
4. The provisions relating to the procedure for appeals, revision, etc.
5. The provisions relating to the procedure for making rules.
6. The provisions relating to the procedure for making regulations.
7. The provisions relating to the grant of exemption.
8. The provisions relating to the levy and collection of interest, penalties and other sums.
9. The provisions relating to the grant of relief.
10. The provisions relating to the application of provisions of other laws.
11. The provisions relating to the application of provisions of this Act to a person not ordinarily resident in India.
12. The provisions relating to the application of provisions of this Act to a foreign company.
13. The
